### Parse .dat file from uniprot to extract all proteins and their sequences according to the following criteria: 
 - at least one molecular function annotation with an experimental evidence code
 - no ambigious amino acids
 - less than 1002 amino acids per protein

In [1]:
import pandas as pd

def evidence(code):
    if (code == "EXP") or (code == "IDA") or (code == "IPI") or (code == "IMP") or (code == "IGI") or (code == "IEP") or (code == "TAS") or (code == "IC"):
        return True
    else:
        return False

In [3]:
with open("uniprot_sprot.dat") as file:
    df = pd.DataFrame(columns=['UniProt_ID', 'Go_terms', 'Seq'])
    
    binding_c = 0
    
    for line in file:
        if line[:2] == "AC":
            name = line[5:].split(";")[0]
            seq = ""
            terms = []
        elif (line[:2] == "DR") and (line[:8]=="DR   GO;"):
            dbR = line.split(";")
            if (dbR[2][1]=="F") and evidence(dbR[3][1:4]):
                if dbR[1].split()[0] != "GO:0005515":
                    terms.append(dbR[1].split()[0])
                else:
                    binding_c += 1
        elif (line[:2] == "  ") and (len(terms)>=1):
            seq += "".join(line.split(" ")).strip()
        elif line[:2] == "//":
            if len(terms)>=1 and len(seq) <= 1002:
                df = df.append({'UniProt_ID':name.strip(),'Go_terms':",".join(terms),'Seq':seq}, ignore_index=True)

In [5]:
for i, seq in enumerate(df["Seq"]):
    if ("B" in seq) or ("O" in seq) or ("J" in seq) or ("U" in seq) or ("X" in seq) or ("Z" in seq):
        df = df.drop(i)

In [6]:
df = df.reset_index()

In [7]:
df.to_pickle("Dataset.pkl")